In [1]:
import re
import torch

In [34]:
IMAGE_SIZE = 24

def read_file(filename):
  with open(filename, 'r') as file:
    # print(file)
    
    lines = file.readlines()
    # print(type(lines))
    
    # Remove newLines
    for i, line in enumerate(lines):
      lines[i] = line.replace('\n', '')
    
    examples = list()
    
    # print(lines[4])
    # print(len(lines))
    
    # We assume these are integers
    EXAMPLES_NR = int(lines[0])
    PIXELS_NR = int(lines[1])
    
    examples_raw = lines[2:EXAMPLES_NR]
    
    # print(PIXELS_NR, type(PIXELS_NR))
    
    for i, example_raw in enumerate(examples_raw):
      # Split by spaces (treats multiple as one)
      tokens = re.split('\s+', example_raw)
      
      pixel_values = tokens[0:PIXELS_NR]
      attributes = tokens[PIXELS_NR:-1]       
        
      for j, el in enumerate(pixel_values):
        pixel_values[j] = float(el)
        
      pixel_values = torch.Tensor(pixel_values).reshape([IMAGE_SIZE, IMAGE_SIZE])
      
      if i == 0:
        print(pixel_values)
        print(pixel_values.size())
      
    
    

In [35]:
read_file('./data/x24x24.txt')

[1.196903, 1.568868, 0.955941, 0.896877, 1.182619, 0.615362, 0.639278, 0.371489, 0.243537, 0.455013, 0.789091, 0.942082, 0.854517, 0.970617, 0.915382, 0.433225, 0.351548, 0.245484, 0.23013, 0.265279, 0.346557, 0.529239, 0.479899, 0.492502, 0.710479, 0.537116, 0.630098, 0.397531, 0.391715, 0.239921, 0.219482, 0.183828, 0.179095, 0.22577, 0.351708, 0.486752, 0.748898, 0.830176, 0.651036, 0.177613, 0.193792, 0.189318, 0.210847, 0.23895, 0.225397, 0.344097, 0.265997, 0.417472, 0.65872, 1.223803, 1.135951, 0.982575, 0.816264, 0.505776, 0.291341, 0.213892, 0.185244, 0.209917, 0.258765, 0.323864, 0.54577, 0.610138, 0.264614, 0.211393, 0.231333, 0.253161, 0.329707, 0.485616, 0.62173, 0.94944, 0.85574, 0.243138, 1.055019, 1.306729, 0.9228, 0.533021, 0.343074, 0.299902, 0.231373, 0.244813, 0.223357, 0.198684, 0.31866, 0.462578, 0.577376, 0.551832, 0.384258, 0.23921, 0.218299, 0.273029, 0.239608, 0.213234, 0.25142, 0.311242, 0.437359, 0.450394, 0.606795, 0.284056, 0.231705, 0.145675, 0.133751, 0.